# Hedditch Authentication and Data Loading

In [ ]:
import requests
import pandas as pd

In [ ]:
# personal use script, secret token
auth = requests.auth.HTTPBasicAuth( 'JNpGyLI-oJ_drb93Dykw9g', 'KgImJaXURcpJG4Ud9-HLBsj8UnnnRQ' )

data = {'grant_type': 'password',
        'username': 'shawwquille_aww_neal',
        'password': 'rampat11' }

headers = {'User-Agent': 'SquatBotTUM/0.0.1' }

res = requests.post( 'https://www.reddit.com/api/v1/access_token', auth=auth, data=data, headers=headers )

token1 = res.json()['access_token']

headers = {**headers, **{'Authorization': f"bearer {token1}"}}

requests.get( 'https://oauth.reddit.com/api/v1/me', headers=headers )

<Response [200]>

In [ ]:
res = requests.get( "https://oauth.reddit.com/r/stopdrinking/new", headers=headers, params={'limit': '100000'} )

print( res.json() )

{'kind': 'Listing', 'data': {'after': 't3_utl47l', 'dist': 100, 'modhash': None, 'geo_filter': '', 'children': [{'kind': 't3', 'data': {'approved_at_utc': None, 'subreddit': 'stopdrinking', 'selftext': 'I’m sure many people here have explored alcohol replacements (sparkling water, tea, NA beer) but I wanted to share about Kin Euphorics, which advertise themselves as alcohol “replacements”. It has plenty of feel-good ingredients (GABA, L-theanime, 5-HTP) and caffiene (or melatonin, depending on which you get) and after having it each day for months now, I have to say that I believe it does uplift my mood. I find that I personally am satisfied by taking a “shot” of something, which I do with Kin. It also comes in can form. Give it a try if you’re interested!', 'author_fullname': 't2_55lksf3w', 'saved': False, 'mod_reason_title': None, 'gilded': 0, 'clicked': False, 'title': 'Kin: my alcohol “replacement”', 'link_flair_richtext': [], 'subreddit_name_prefixed': 'r/stopdrinking', 'hidden': 

In [ ]:
df = pd.DataFrame()

for post in res.json()['data']['children']:
  df = df.append({
      'subreddit': post['data']['subreddit'],
      'title': post['data']['title'],
      'selftext': post['data']['selftext'],
      'upvote_ratio': post['data']['upvote_ratio'],
      'ups': post['data']['ups'],
      'downs': post['data']['downs'],
      'score': post['data']['score']
  }, ignore_index=True )

In [ ]:
df.head()

,subreddit,title,selftext,upvote_ratio,ups,downs,score
0,stopdrinking,Kin: my alcohol “replacement”,I’m sure many people here have explored alcoho...,1.0,3.0,0.0,3.0
1,stopdrinking,6 days sober and I need some suggestions,"That title makes zero sense, but essentially I...",1.0,2.0,0.0,2.0
2,stopdrinking,Dating again sober,Tomorrow will be two months sober and I will b...,1.0,2.0,0.0,2.0
3,stopdrinking,Partner is out of town - really wanted to drin...,My partner is out of the country for a work tr...,1.0,3.0,0.0,3.0
4,stopdrinking,10 Days into my 30 Day sober challenge.,Been drinking around 10-12 drinks a night for ...,1.0,3.0,0.0,3.0


In [ ]:
raw_text1 = res.json()['data']['children'][2]['data']['selftext']

print( raw_text1 )

Tomorrow will be two months sober and I will be going on my second date.  The first date she asked me if I drink and I said “no” without getting into to much detail about why I don’t.  She was very respectful about me being sober and she has no complaints of being with someone sober.  She is the first date I have been on sober and it was easy; I acted myself, smiled and laughed and we hit it off good.  IWNDWYT


# Named Entity Recognition

### 1. spaCy

In [ ]:
import spacy
from spacy import displacy
import spacy.cli

spacy.cli.download("en_core_web_lg")

NER = spacy.load( "en_core_web_lg" )

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
# [ "Germany", "'s", "representative", "to", "the", "European", "Union", "'s", "veterinary", "committee", "Werner", "Zwingmann", "said", "on", "Wednesday", "consumers", "should", "buy", "sheepmeat", "from", "countries", "other", "than", "Britain", "until", "the", "scientific", "advice", "was", "clearer", "." ]

raw_text1 = "Germany's representative to the EUropean Union's veterinary committee, Werner Zwingmann, said on Wednesday consumers should buy sheep meat from countries other than Britain until the scientific advice was clearer. Two days later, on Tuesday, another company from Korea named Samsung also decided to destroy stuff."
text1 = NER( raw_text1 )

In [ ]:
for word in text1.ents:
  print( word.text, word.label_ )

Germany GPE
the EUropean Union's ORG
Werner Zwingmann PERSON
Wednesday DATE
Britain GPE
Two days later DATE
Tuesday DATE
Korea GPE
Samsung ORG


In [ ]:
displacy.render( text1, style="ent", jupyter=True )

In [ ]:
text1 = NER( raw_text1 )

for word in text1.ents:
  print( word.text, word.label_ )

Tomorrow DATE
two months DATE
second ORDINAL
first ORDINAL
first ORDINAL
IWNDWYT ORG


In [ ]:
displacy.render( text1, style="ent", jupyter=True )

### 2. GATE (Annie)

In [ ]:
import string

url_new = "https://cloud-api.gate.ac.uk/process/annie-named-entity-recognizer"

special_characters = string.punctuation + "’" + "“" + "”" + "❤️"

annie_text1 = ''.join( i for i in raw_text1 if i not in special_characters )

print( annie_text1 )

headers = {'Content-Type': 'text/plain'}
response = requests.post( url_new, data=annie_text1, headers=headers ).json()

import json
print( json.dumps( response, indent=2 ) )

Tomorrow will be two months sober and I will be going on my second date  The first date she asked me if I drink and I said no without getting into to much detail about why I dont  She was very respectful about me being sober and she has no complaints of being with someone sober  She is the first date I have been on sober and it was easy I acted myself smiled and laughed and we hit it off good  IWNDWYT
{
  "text": "Tomorrow will be two months sober and I will be going on my second date  The first date she asked me if I drink and I said no without getting into to much detail about why I dont  She was very respectful about me being sober and she has no complaints of being with someone sober  She is the first date I have been on sober and it was easy I acted myself smiled and laughed and we hit it off good  IWNDWYT",
  "entities": {}
}


### 3. NLTK

In [ ]:
import nltk
nltk.download( 'punkt' )
nltk.download( 'averaged_perceptron_tagger' )
nltk.download( 'maxent_ne_chunker' )
nltk.download( 'words' )
{( ' '.join( c[0] for c in chunk ), chunk.label() ) for chunk in nltk.ne_chunk( nltk.pos_tag( nltk.word_tokenize( raw_text1 ) ) ) if hasattr( chunk, 'label' )}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


{('IWNDWYT', 'ORGANIZATION'), ('Tomorrow', 'GPE')}

### 4. Stanford NER

In [ ]:
!pip3 install nltk==3.2.4
!wget http://nlp.stanford.edu/software/stanford-ner-2015-04-20.zip
!unzip stanford-ner-2015-04-20.zip

from nltk.tag.stanford import StanfordNERTagger

jar = "stanford-ner-2015-04-20/stanford-ner-3.5.2.jar"
model = "stanford-ner-2015-04-20/classifiers/" 

st_3class = StanfordNERTagger( model + "english.all.3class.distsim.crf.ser.gz", jar, encoding='utf8' ) 
st_4class = StanfordNERTagger( model + "english.conll.4class.distsim.crf.ser.gz", jar, encoding='utf8' ) 
st_7class = StanfordNERTagger( model + "english.muc.7class.distsim.crf.ser.gz", jar, encoding='utf8' )

     |████████████████████████████████| 1.2 MB 5.2 MB/s 
  Created wheel for nltk: filename=nltk-3.2.4-py3-none-any.whl size=1367723 sha256=21df0fa4fc443f3ff7414859d368cf72bb152eba97bdced3d3de0a421d4fb6f2
  Stored in directory: /root/.cache/pip/wheels/90/5e/9e/4cb46185f2a16c60e6fc524372ba7fef89ce3347734c8798b6
Successfully built nltk
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


--2022-05-20 18:50:38--  http://nlp.stanford.edu/software/stanford-ner-2015-04-20.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/software/stanford-ner-2015-04-20.zip [following]
--2022-05-20 18:50:38--  https://nlp.stanford.edu/software/stanford-ner-2015-04-20.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://downloads.cs.stanford.edu/nlp/software/stanford-ner-2015-04-20.zip [following]
--2022-05-20 18:50:38--  https://downloads.cs.stanford.edu/nlp/software/stanford-ner-2015-04-20.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response..

/usr/local/lib/python3.7/dist-packages/nltk/tag/stanford.py:183: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  '-tokenizerFactory',
/usr/local/lib/python3.7/dist-packages/nltk/tag/stanford.py:183: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  '-tokenizerFactory',
/usr/local/lib/python3.7/dist-packages/nltk/tag/stanford.py:183: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  '-tokenizerFactory',


In [ ]:
st_3class.tag( raw_text1.split() )
# st_4class.tag( raw_text1.split() )
# st_7class.tag( raw_text1.split() )

[('Tomorrow', 'O'),
 ('will', 'O'),
 ('be', 'O'),
 ('two', 'O'),
 ('months', 'O'),
 ('sober', 'O'),
 ('and', 'O'),
 ('I', 'O'),
 ('will', 'O'),
 ('be', 'O'),
 ('going', 'O'),
 ('on', 'O'),
 ('my', 'O'),
 ('second', 'O'),
 ('date.', 'O'),
 ('The', 'O'),
 ('first', 'O'),
 ('date', 'O'),
 ('she', 'O'),
 ('asked', 'O'),
 ('me', 'O'),
 ('if', 'O'),
 ('I', 'O'),
 ('drink', 'O'),
 ('and', 'O'),
 ('I', 'O'),
 ('said', 'O'),
 ('“no”', 'O'),
 ('without', 'O'),
 ('getting', 'O'),
 ('into', 'O'),
 ('to', 'O'),
 ('much', 'O'),
 ('detail', 'O'),
 ('about', 'O'),
 ('why', 'O'),
 ('I', 'O'),
 ('don’t.', 'O'),
 ('She', 'O'),
 ('was', 'O'),
 ('very', 'O'),
 ('respectful', 'O'),
 ('about', 'O'),
 ('me', 'O'),
 ('being', 'O'),
 ('sober', 'O'),
 ('and', 'O'),
 ('she', 'O'),
 ('has', 'O'),
 ('no', 'O'),
 ('complaints', 'O'),
 ('of', 'O'),
 ('being', 'O'),
 ('with', 'O'),
 ('someone', 'O'),
 ('sober.', 'O'),
 ('She', 'O'),
 ('is', 'O'),
 ('the', 'O'),
 ('first', 'O'),
 ('date', 'O'),
 ('I', 'O'),
 ('have', '

### 5. Polyglot

In [ ]:
!pip3 install -U git+https://github.com/aboSamoor/polyglot.git@master
!polyglot download embeddings2.en ner2.en

from polyglot.text import Text

  Cloning https://github.com/aboSamoor/polyglot.git (to revision master) to /tmp/pip-req-build-qx1pjmvb
  Running command git clone -q https://github.com/aboSamoor/polyglot.git /tmp/pip-req-build-qx1pjmvb
     |████████████████████████████████| 41.4 MB 1.3 MB/s 
     |████████████████████████████████| 305 kB 56.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for polyglot: filename=polyglot-16.7.4-py2.py3-none-any.whl size=70670 sha256=d61f0ea28c8c37610cddec86d0f9d9760d91fc183068b812ec10ad06c40cc380
  Stored in directory: /tmp/pip-ephem-wheel-cache-rak94_q9/wheels/6d/b6/83/526ac20beb2e8b06c33aea6ac5edf6a07d3de55011bcc35ee8
  Created wheel for pycld2: filename=pycld2-0.41-cp37-cp37m-linux_x86_64.whl size=9834169 sha256=8039e9470af58a79fd9d549839382e4ac95ac38bd858f427f1ef356f2a315366
  Stored in directory: /root/.cache/pip/wheels/ed/e4/58/ed2e9f43c07d617cc81fe7aff0fc6e42b16c9cf6afe960b614

[polyglot_data] Downloading package embeddings2.en to
[polyglot_data]     /root/polyglot_data...
[polyglot_data] Downloading package ner2.en to /root/polyglot_data...


In [ ]:
Text(raw_text1).entities

[I-PER(['IWNDWYT'])]

### 6. Flair

In [ ]:
!pip3 install flair
!pip3 install sacremoses

from flair.models import SequenceTagger

model = SequenceTagger.load('ner-ontonotes-fast') #.load('ner')

from flair.data import Sentence

     |████████████████████████████████| 401 kB 5.0 MB/s 
     |████████████████████████████████| 84 kB 1.9 MB/s 
     |████████████████████████████████| 788 kB 33.1 MB/s 
     |████████████████████████████████| 1.2 MB 31.5 MB/s 
     |████████████████████████████████| 1.6 MB 43.1 MB/s 
     |████████████████████████████████| 46 kB 3.2 MB/s 
     |████████████████████████████████| 53 kB 1.6 MB/s 
     |████████████████████████████████| 4.2 MB 40.9 MB/s 
     |████████████████████████████████| 19.7 MB 53.4 MB/s 
     |████████████████████████████████| 981 kB 57.2 MB/s 
     |████████████████████████████████| 199 kB 47.1 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 6.6 MB 42.8 MB/s 
     |████████████████████████████████| 596 kB 37.7 MB/s 
  Created wheel for mpld3: filename=mpld3-0.3-py3-none-any.whl size=116702 sha256=0b35d53fb1d892e9e8a4feacdff463c3e045e450b034a8e6f7f34ca4fd845dc3
  Stored in directory: /root/.cache/pip/wheels/26

     |████████████████████████████████| 880 kB 5.4 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=536f10a5523488be7640be1c9f3bad60a2a059140c9a4373ffe3a232aaac1e65
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses


Downloading:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

2022-05-20 18:59:46,156 loading file /root/.flair/models/ner-english-ontonotes-fast/0d55dd3b912da9cf26e003035a0c269a0e9ab222f0be1e48a3bbba3a58c0fed0.c9907cd5fde3ce84b71a4172e7ca03841cd81ab71d13eb68aa08b259f57c00b6
2022-05-20 18:59:59,439 SequenceTagger predicts: Dictionary with 76 tags: <unk>, O, B-CARDINAL, E-CARDINAL, S-PERSON, S-CARDINAL, S-PRODUCT, B-PRODUCT, I-PRODUCT, E-PRODUCT, B-WORK_OF_ART, I-WORK_OF_ART, E-WORK_OF_ART, B-PERSON, E-PERSON, S-GPE, B-DATE, I-DATE, E-DATE, S-ORDINAL, S-LANGUAGE, I-PERSON, S-EVENT, S-DATE, B-QUANTITY, E-QUANTITY, S-TIME, B-TIME, I-TIME, E-TIME, B-GPE, E-GPE, S-ORG, I-GPE, S-NORP, B-FAC, I-FAC, E-FAC, B-NORP, E-NORP, S-PERCENT, B-ORG, E-ORG, B-LANGUAGE, E-LANGUAGE, I-CARDINAL, I-ORG, S-WORK_OF_ART, I-QUANTITY, B-MONEY


In [ ]:
s = Sentence(raw_text1)
model.predict(s)
s.to_dict(tag_type='ner')

{'ner': [{'confidence': 0.9578028917312622, 'value': 'DATE'},
  {'confidence': 0.9192259609699249, 'value': 'DATE'},
  {'confidence': 0.9988188147544861, 'value': 'ORDINAL'},
  {'confidence': 0.9984812140464783, 'value': 'ORDINAL'},
  {'confidence': 0.9679218530654907, 'value': 'ORDINAL'},
  {'confidence': 0.7743558883666992, 'value': 'PERSON'}],
 'text': 'Tomorrow will be two months sober and I will be going on my second date.  The first date she asked me if I drink and I said “no” without getting into to much detail about why I don’t.  She was very respectful about me being sober and she has no complaints of being with someone sober.  She is the first date I have been on sober and it was easy; I acted myself, smiled and laughed and we hit it off good.  IWNDWYT'}

### 7. AllenNLP

In [ ]:
!pip3 install allennlp
!pip3 install --upgrade google-cloud-storage

from allennlp.predictors import Predictor

ImportError: ignored

In [ ]:
al = Predictor.from_path("https://s3-us-west-2.amazonaws.com/allennlp/models/ner-model-2020.02.10.tar.gz")
al.predict(sentence=raw_text1)

NameError: ignored

### 8. BERT Base NER

In [ ]:
!pip install datasets
!pip install tokenizers
!pip install transformers

     |████████████████████████████████| 346 kB 5.1 MB/s 
     |████████████████████████████████| 1.1 MB 45.8 MB/s 
     |████████████████████████████████| 212 kB 41.0 MB/s 
     |████████████████████████████████| 140 kB 51.3 MB/s 
     |████████████████████████████████| 271 kB 48.9 MB/s 
     |████████████████████████████████| 94 kB 3.1 MB/s 
     |████████████████████████████████| 144 kB 49.2 MB/s 


In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)

Downloading:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/829 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/413M [00:00<?, ?B/s]

In [ ]:
ner_results = nlp( raw_text1 )
print(ner_results)

[]
